LangChain Tool
```
Tool : llm에이전트가 수행할수 있는 외부기능이나 api를 말함(날씨정보나 웹검색,게산기)
이유 : gpt는 최신정보에 접근할수 없다.
구조 : 이름 설명 함수실행로직
툴 호출 방식 :  GPT-4 모델들은 OpenAI의 함수호출 기능을 통해 툴을 직접 호출할수 있다다
```
외부 API 호출 Tool 구현(날씨API, 뉴스API)

In [1]:
%pip install langchain-core langchain-openai openai python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
# .env 로드
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ["OPEN_WEATHER_MAP"][:5])
print(os.environ["NEWSAPI_API_KEY"][:5])

83e83
7ca03


In [42]:
import requests
from pydantic import BaseModel, Field

# 입력 스키마 정의
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름 (영문)")

def get_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 반환합니다."""
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",      # 섭씨 온도
        "lang": "kr"            # 한국어 응답 (설명이 한국어로 오도록)
    }
    response = requests.get(url, params=params)
    data = response.json()
    # API 응답에 따른 처리
    if data.get("cod") != 200:
        # 도시 정보를 찾지 못한 경우
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."
    # 필요한 정보 파싱
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    city_name = data["name"]  # 응답에 있는 도시 이름 (영문일 수 있음)
    return f"{city_name}의 현재 기온은 {temp}℃, 날씨는 {desc}입니다."

In [4]:
# pydantic 모델을 사용하여 입력값 검증
# class WeatherInput(BaseModel):
#     city : str = Field(description="날씨를 조회할 도시 이름(영문)")

# class MyClass:
#     def __init__(self, ab: str):
#         self.city = ab

# MyClass(ab=100)  # 사용자가 의도한 문자열이 전달되지 않아도 python은 타입에러를 발생시키지 않음
# result = WeatherInput( city=100)    # pydantic 모델은 타입에러를 발생시킴
# result.city

In [5]:
get_weather('seoul')

'seoul의 현재 기온은 25.76도이며, 날씨는 약간의 구름이 낀 하늘입니다.'

뉴스검색

In [43]:
class NewsInput(BaseModel):
    keyword: str = Field(description="뉴스를 조회할 키워드")
    
def get_news(keyword:str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "apiKey": api_key,
        "language": "ko",
        "sortBy": "publishedAt",  #최신뉴스
        "pageSize": 1  # 최근 1건
    }
    response = requests.get(url, params=params)
    data = response.json()    
    articles = data.get('articles')
    if not articles:
        return "해당 키워드에 대한 뉴스가 없습니다."
    
    # 가장 첫번째 뉴스선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스: "{title}" (출처: {source})'

In [7]:
print(get_news('취업'))

취업에 대한 최신 뉴스: "울산시, '광역 비자 시범 대상지'로 최종 선정" (출처: Ohmynews.com)


LangChain Tool 객체로 변환
```
get_weather, get_news 함수를 Tool로 등록
```

In [44]:
from langchain_core.runnables import RunnableLambda

# 날씨 함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda input: get_weather(input["city"]))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구",
    args_schema=WeatherInput
)

# 뉴스 함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda input: get_news(input["keyword"]))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구",
    args_schema=NewsInput
)

print(f"Tool 이름: {weather_tool.name}, 설명: {weather_tool.description}")
print(f"Tool 이름: {news_tool.name}, 설명: {news_tool.description}")

Tool 이름: get_weather, 설명: 도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구
Tool 이름: get_news, 설명: 키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구


In [36]:
# 툴없이 
from langchain_openai import ChatOpenAI
# 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 출력 파서
from langchain_core.output_parsers import StrOutputParser
user_question = '서울 날씨 알려줘'
# 사용자 질문을 처리하는 프롬프트 템플릿
user_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 날씨와 뉴스 정보를 제공하는 AI입니다."),
    ("human", "{question}")
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = user_prompt | llm | StrOutputParser()
# 사용자 질문을 처리하는 체인 실행
result = chain.invoke({"question": user_question})
print(result)


현재 서울의 날씨 정보를 실시간으로 제공할 수는 없지만, 일반적으로 10월의 서울 날씨는 선선하고 맑은 날이 많습니다. 평균 기온은 약 10도에서 20도 사이이며, 가끔 비가 내릴 수도 있습니다. 정확한 날씨 정보를 원하신다면 기상청 웹사이트나 날씨 앱을 확인해 보시기 바랍니다.


LLM 최종 답변 생성 요청

In [37]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 프롬프트구성 : 시스템메시지 + 사용자 질문 + 결과를 맥락으로 제공
weather_info = get_weather('seoul')
system_prompt = "당신은 유용한 AI 어시스턴트입니다. 사용자 질문에 맞게 제공된 정보를 활용해 답변하세요"
tool_info_prompt = f'도구가 제공한 추가 정보:{weather_info}'
ask_prompt = '위 정보를 참고해서 사용자 질문에 답변하세요'
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question),
    AIMessage(content=tool_info_prompt),  #도구 결과를 마치 ai의 답변처럼 추가가
    HumanMessage(content=ask_prompt)
]
result = llm(messages)   # LLM에 메시지 전달
print(result.content)

현재 서울의 날씨는 약간의 구름이 낀 상태이며, 기온은 25.76도입니다. 외출 시 가벼운 옷차림이 적합할 것 같습니다. 추가적인 날씨 정보가 필요하시면 말씀해 주세요!


여러 tool을 연결하도록 에이전트 구성

In [45]:
from langchain.agents import initialize_agent, AgentType

# 앞서 만든 Tool 객체 리스트 준비
tools = [weather_tool, news_tool]

# OpenAI 함수 기반 에이전트 생성
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True  # intermediate 단계 로그를 출력하여 확인
)

In [46]:
query1 = "서울의 날씨와 AI 관련 최신 뉴스를 알려줘."
result1 = agent.run(query1)
print("Agent Result 1:", result1)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Seoul'}`


'Seoul'의 날씨 정보를 찾을 수 없습니다.
Invoking: `get_news` with `{'keyword': 'AI'}`


AI에 대한 최신 뉴스: "블룸버그, 카메라 탑재 애플워치 개발 계획 '백지화'" (출처: Kbench.com)서울의 날씨 정보를 찾을 수 없었습니다. 대신 AI 관련 최신 뉴스는 다음과 같습니다:

- **"블룸버그, 카메라 탑재 애플워치 개발 계획 '백지화'"** (출처: Kbench.com)

더 궁금한 점이 있으면 말씀해 주세요!

> Finished chain.
Agent Result 1: 서울의 날씨 정보를 찾을 수 없었습니다. 대신 AI 관련 최신 뉴스는 다음과 같습니다:

- **"블룸버그, 카메라 탑재 애플워치 개발 계획 '백지화'"** (출처: Kbench.com)

더 궁금한 점이 있으면 말씀해 주세요!
